In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.ml.feature import *

In [3]:
df = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       parquet("final.parquet")

df.show(vertical=True)

-RECORD 0---------------------------------------------------------------------------
 features_smile.msisdn                                       | 22506138989          
 features_smile.nombre_sms_sortants                          | 2                    
 features_smile.nombre_correspondant_sms_sortant             | 1                    
 features_smile.nombre_appels_sortants                       | 0                    
 features_smile.nombre_correspondant_appels_sortant          | 2                    
 features_smile.nombre_correspondant_distinct_appels_sortant | 1                    
 features_smile.duree_totale_appel_sortant                   | 0                    
 features_smile.duree_moyenne_appel_sortant                  | 0.0                  
 features_smile.nombre_sms_entrants                          | 5                    
 features_smile.nombre_correspondant_sms_entrant             | 3                    
 features_smile.nombre_appel_entrants                        | 4 

In [4]:
df1 = df
df1.show(vertical = True)

-RECORD 0---------------------------------------------------------------------------
 features_smile.msisdn                                       | 22506138989          
 features_smile.nombre_sms_sortants                          | 2                    
 features_smile.nombre_correspondant_sms_sortant             | 1                    
 features_smile.nombre_appels_sortants                       | 0                    
 features_smile.nombre_correspondant_appels_sortant          | 2                    
 features_smile.nombre_correspondant_distinct_appels_sortant | 1                    
 features_smile.duree_totale_appel_sortant                   | 0                    
 features_smile.duree_moyenne_appel_sortant                  | 0.0                  
 features_smile.nombre_sms_entrants                          | 5                    
 features_smile.nombre_correspondant_sms_entrant             | 3                    
 features_smile.nombre_appel_entrants                        | 4 

In [17]:
df1 = df1.na.fill(0)
df1.show(vertical = True)

-RECORD 0---------------------------------------------------------------------------
 features_smile_msisdn                                       | 22506138989          
 features_smile_nombre_sms_sortants                          | 2.0                  
 features_smile_nombre_correspondant_sms_sortant             | 1.0                  
 features_smile_nombre_appels_sortants                       | 0.0                  
 features_smile_nombre_correspondant_appels_sortant          | 2.0                  
 features_smile_nombre_correspondant_distinct_appels_sortant | 1.0                  
 features_smile_duree_totale_appel_sortant                   | 0.0                  
 features_smile_duree_moyenne_appel_sortant                  | 0.0                  
 features_smile_nombre_sms_entrants                          | 5.0                  
 features_smile_nombre_correspondant_sms_entrant             | 3.0                  
 features_smile_nombre_appel_entrants                        | 4.

In [22]:
liste = [
 'features_smile_nombre_sms_sortants',
 'features_smile_nombre_correspondant_sms_sortant',
 'features_smile_nombre_appels_sortants',
 'features_smile_nombre_correspondant_appels_sortant',
 'features_smile_nombre_correspondant_distinct_appels_sortant',
 'features_smile_duree_totale_appel_sortant',
 'features_smile_duree_moyenne_appel_sortant',
 'features_smile_nombre_sms_entrants',
 'features_smile_nombre_correspondant_sms_entrant',
 'features_smile_nombre_appel_entrants',
 'features_smile_nombre_correspondant_appel_entrant',
 'features_smile_nombre_correspondant_appel_distinct_entrant',
 'features_smile_duree_totale_appel_entrant',
 'features_smile_duree_moyenne_appel_entrant',
 'features_smile_nombre_sms',
 'features_smile_nombre_appels',
 'features_smile_nombre_correspondants_sms',
 'features_smile_nombre_correspondants',
 'features_smile_nombre_correspondants_distincts',
 'features_smile_nombre_correspondants_distincts_appels',
 'features_smile_nombre_correspondants_appels',
 'features_smile_duree_totale_appels',
 'features_smile_duree_moyenne_appels',
 'features_smile_taux_inities_sms',
 'features_smile_nombre_appels_onnet',
 'features_smile_taux_appels_inities_onnet',
 'features_smile_nombre_appels_offnet',
 'features_smile_taux_appels_inities_offnet',
 'features_smile_nombre_appels_inter',
 'features_smile_taux_appels_inities_inter',
 'features_smile_nombre_appels_complice',
 'features_smile_taux_appels_inities_complice',
 'features_smile_nbre_imsi',
 'features_smile_nbre_distinct_imei',
 'features_smile_ratio_nombre_imsi_imei',
 'features_smile_nombre_appels_nuit',
 'features_smile_nombre_appels_nuit_distinct',
 'features_smile_temps_moyen_evenement',
 'features_smile_variance_evenement',
 'features_smile_temps_moyen_appel',
 'features_smile_variance_appel',
 'features_smile_temps_moyen_sms',
 'features_smile_variance_sms',
 'features_smile_totaldureeusedata',
 'features_smile_nbre_sim',
 'features_smile_supplier_name',
 'features_smile_nombre_cal_onnet',
 'features_smile_nbre_moy_cal_onnet',
 'features_smile_duree_moy_cal_onnet',
 'features_smile_duree_cal_onnet',
 'features_smile_nbre_moy_cal_offnet',
 'features_smile_nombre_cal_offnet',
 'features_smile_duree_moy_cal_offnet',
 'features_smile_duree_cal_offnet',
 'features_smile_duree_mean_call_out',
 'features_smile_ratio_corresp_out_in',
 'features_smile_ratio_call_out_in',
 'features_smile_numb_inter_correp_out_distinct',
 'features_smile_numb_inter_correp_out',
 'features_smile_time_call_out',
 'features_smile_numb_call_inter_out',
 'features_smile_numb_inter_correp_in_distinct',
 'features_smile_numb_inter_correp_in',
 'features_smile_time_call_in',
 'features_smile_numb_call_inter_in',
 'features_smile_numb_inter_corresp_distinct',
 'features_smile_numb_inter_correp',
 'features_smile_numb_call_inter',
 'features_pass_souscription_profil_nombre_profil',
 'features_pass_souscription_profil_nombre_fois_profil_change',
 'features_pass_souscription_profil_nombre_souscription',
 'features_pass_souscription_profil_nombre_pass_achete']

In [23]:
df1 = df1.toDF(*(c.replace('.', '_') for c in df1.columns))

In [24]:
for cl in liste:
    df1 = df1.withColumn(cl, col(cl).cast(DoubleType()))

In [25]:
assemble=VectorAssembler(inputCols=[
 'features_smile_nombre_sms_sortants',
 'features_smile_nombre_correspondant_sms_sortant',
 'features_smile_nombre_appels_sortants',
 'features_smile_nombre_correspondant_appels_sortant',
 'features_smile_nombre_correspondant_distinct_appels_sortant',
 'features_smile_duree_totale_appel_sortant',
 'features_smile_duree_moyenne_appel_sortant',
 'features_smile_nombre_sms_entrants',
 'features_smile_nombre_correspondant_sms_entrant',
 'features_smile_nombre_appel_entrants',
 'features_smile_nombre_correspondant_appel_entrant',
 'features_smile_nombre_correspondant_appel_distinct_entrant',
 'features_smile_duree_totale_appel_entrant',
 'features_smile_duree_moyenne_appel_entrant',
 'features_smile_nombre_sms',
 'features_smile_nombre_appels',
 'features_smile_nombre_correspondants_sms',
 'features_smile_nombre_correspondants',
 'features_smile_nombre_correspondants_distincts',
 'features_smile_nombre_correspondants_distincts_appels',
 'features_smile_nombre_correspondants_appels',
 'features_smile_duree_totale_appels',
 'features_smile_duree_moyenne_appels',
 'features_smile_taux_inities_sms',
 'features_smile_nombre_appels_onnet',
 'features_smile_taux_appels_inities_onnet',
 'features_smile_nombre_appels_offnet',
 'features_smile_taux_appels_inities_offnet',
 'features_smile_nombre_appels_inter',
 'features_smile_taux_appels_inities_inter',
 'features_smile_nombre_appels_complice',
 'features_smile_taux_appels_inities_complice',
 'features_smile_nbre_imsi',
 'features_smile_nbre_distinct_imei',
 'features_smile_ratio_nombre_imsi_imei',
 'features_smile_nombre_appels_nuit',
 'features_smile_nombre_appels_nuit_distinct',
 'features_smile_temps_moyen_evenement',
 'features_smile_variance_evenement',
 'features_smile_temps_moyen_appel',
 'features_smile_variance_appel',
 'features_smile_temps_moyen_sms',
 'features_smile_variance_sms',
 'features_smile_totaldureeusedata',
 'features_smile_nbre_sim',
 'features_smile_supplier_name',
 'features_smile_nombre_cal_onnet',
 'features_smile_nbre_moy_cal_onnet',
 'features_smile_duree_moy_cal_onnet',
 'features_smile_duree_cal_onnet',
 'features_smile_nbre_moy_cal_offnet',
 'features_smile_nombre_cal_offnet',
 'features_smile_duree_moy_cal_offnet',
 'features_smile_duree_cal_offnet',
 'features_smile_duree_mean_call_out',
 'features_smile_ratio_corresp_out_in',
 'features_smile_ratio_call_out_in',
 'features_smile_numb_inter_correp_out_distinct',
 'features_smile_numb_inter_correp_out',
 'features_smile_time_call_out',
 'features_smile_numb_call_inter_out',
 'features_smile_numb_inter_correp_in_distinct',
 'features_smile_numb_inter_correp_in',
 'features_smile_time_call_in',
 'features_smile_numb_call_inter_in',
 'features_smile_numb_inter_corresp_distinct',
 'features_smile_numb_inter_correp',
 'features_smile_numb_call_inter',
 'features_pass_souscription_profil_nombre_profil',
 'features_pass_souscription_profil_nombre_fois_profil_change',
 'features_pass_souscription_profil_nombre_souscription',
 'features_pass_souscription_profil_nombre_pass_achete'], outputCol='features')

In [26]:
assembled_data = assemble.transform(df1)

In [27]:
scale = StandardScaler(inputCol='features',outputCol='standardized')

data_scale = scale.fit(assembled_data)

data_scale_output = data_scale.transform(assembled_data)
#data_scale_output.show(2)

In [28]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo = KMeans(featuresCol='standardized', k=2)
    
KMeans_fit = KMeans_algo.fit(data_scale_output)
    
output = KMeans_fit.transform(data_scale_output)

score = evaluator.evaluate(output)

silhouette_score.append(score)

print("Silhouette Score:",score)

Silhouette Score: 0.7345763674449337


In [30]:
output.show(vertical = True)

-RECORD 0---------------------------------------------------------------------------
 features_smile_msisdn                                       | 22506138989          
 features_smile_nombre_sms_sortants                          | 2.0                  
 features_smile_nombre_correspondant_sms_sortant             | 1.0                  
 features_smile_nombre_appels_sortants                       | 0.0                  
 features_smile_nombre_correspondant_appels_sortant          | 2.0                  
 features_smile_nombre_correspondant_distinct_appels_sortant | 1.0                  
 features_smile_duree_totale_appel_sortant                   | 0.0                  
 features_smile_duree_moyenne_appel_sortant                  | 0.0                  
 features_smile_nombre_sms_entrants                          | 5.0                  
 features_smile_nombre_correspondant_sms_entrant             | 3.0                  
 features_smile_nombre_appel_entrants                        | 4.

In [32]:
output.select("prediction").groupBy("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|         1| 245706|
|         0|1516050|
+----------+-------+



In [33]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score1=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo1 = KMeans(featuresCol='standardized', k=3)
    
KMeans_fit1 = KMeans_algo1.fit(data_scale_output)
    
output1 = KMeans_fit1.transform(data_scale_output)

score1 = evaluator.evaluate(output1)

silhouette_score1.append(score1)

print("Silhouette Score:",score1)

Silhouette Score: 0.75305637183942


In [34]:
output1.select("prediction").groupBy("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|         1| 245766|
|         2|    855|
|         0|1515135|
+----------+-------+



In [35]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score2=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo2 = KMeans(featuresCol='standardized', k=4)
    
KMeans_fit2 = KMeans_algo2.fit(data_scale_output)
    
output2 = KMeans_fit2.transform(data_scale_output)

score2 = evaluator.evaluate(output2)

silhouette_score2.append(score2)

print("Silhouette Score:",score2)

Silhouette Score: 0.75305637183942


In [36]:
output2.select("prediction").groupBy("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|         1| 245766|
|         2|    855|
|         0|1515135|
+----------+-------+

